# Data of Kobe·Bryant

## 数据说明

- 数据属性

        G(season game)， Age(age on Feb 1 of the season)， AgeY(age of the year)，Tm(Team)，GL(game location(#: home, @: away))，Opp(opponent)，W/L(win or lose)，PTD(point different)，

        GS(games started)，MP(minutes played)，FG(field goals)，FGA(field goal attempts)，FGP(field goal percentage)，3P(3-point field goals)，3PA(3-point field goal attempts)，

        3PP(3-point field goal percentage)，FT(free throws)，FTA(free throw attempts)，FTP(free throw percentage)，ORB(offensive rebounds)，DRB(defensive rebounds)，TRB(total rebounds)，

        AST(assists)，STL(steals)，BLK(blocks)，TOV(turnovers)，PF(personal fouls)，PTS(points)，GmSc(game score)，+/-(plus/minus)，MPP(minutes played switch)

- 球队缩写
        
        ATL(Atlanta Hawks)，BOS(Boston Celtics)，BRK(Brooklyn Nets)，CHA(Charlotte Bobcats)，CHH(Charlotte Hornets)，CHI(Chicago Bulls)，CHO(Charlotte Hornets)，CLE(Cleveland Cavaliers)，
        
        DAL(Dallas Mavericks)，DEN(Denver Nuggets)，DET(Detroit Pistons)，GSW(Golden State Warriors)，HOU(Houston Rockets)，IND(Indiana Pacers)，LAC(Los Angeles Clippers)，MEM(Memphis Grizzlies)，
        
        MIA(Miami Heat)，MIL(Milwaukee Bucks)，MIN(Minnesota Timberwolves)，NJN(New Jersey Nets)，NOH(New Orleans Hornets)，NOK(New Orleans/Oklahoma City Hornets)，NOP(New Orleans Pelicans)，
        
        NYK(New York Knicks)，OKC(Oklahoma City Thunder)，ORL(Orlando Magic)，PHI(Philadelphia 76ers)，PHO(Phoenix Suns)，POR(Portland Trail Blazers)，SAC(Sacramento Kings)，
        
        SAS(San Antonio Spurs)，SEA(Seattle SuperSonics)，TOR(Toronto Raptors)，UTA(Utah Jazz)，VAN(Vancouver Grizzlies)，WAS(Washington Wizards)，WSB(Washington Bullets)

## 加载库

In [1]:
import os
import re
import json
import time
import requests
import pandas as pd
from datetime import *
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType, ChartType
from bs4 import BeautifulSoup
from selenium import webdriver   # 导入浏览器驱动模块
import warnings
warnings.filterwarnings("ignore")  # 忽略版本问题

# Regular数据收集

## 网站解析及数据获取

In [2]:
r_list = []
def regulars(play_year):
    url = 'https://www.basketball-reference.com/players/b/bryanko01/gamelog/' + str(play_year)
    r = requests.get(url, timeout=30)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, "html.parser")
    d_find = soup.find('div', {'id':'div_pgl_basic'}).find('table')
    for i in range(1,1347):
        hi = []
        tbody = d_find.find('tbody').find('tr', {'id':'pgl_basic.'+str(i)})
        if not tbody:
            continue
        for td in tbody.find_all('td', recursive=False):
            ht = td.text
            hi.append(ht)
#             print(ht)
        r_list.append(hi)
    return r_list

In [3]:
for play_years in range(1997, 2017):
    play_year = play_years
    regulars(play_year)
    print(play_year)

1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [4]:
i_list = ['G', 'Date', 'Age', 'Tm', 'GL', 'Opp', 'GR', 'GS', 'MP', 'FG', 'FGA', 'FG%', 
          '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 
          'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-']


## 数据预处理

In [5]:
f_data = pd.DataFrame(r_list, columns=i_list)
f_data['Date'] = f_data['Date'].map(lambda x: datetime.strptime(x, "%Y-%m-%d"))  # 转换为时间类型
f_data = f_data.set_index('Date')
# 转换为整型数据
f_data[['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']] = f_data[['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].astype(int)

f_data['GL'].replace('', '#', inplace=True)  # 替换主场空数据
# 处理并转换年龄及比赛结果数据
f_data['AgeY'] = f_data['Age'].str.split('-', expand=True)[0].astype(int)
f_data['W/L'] = f_data['GR'].str.split(' ', expand=True)[0]
f_data['PTD'] = f_data['GR'].str.split('\(', expand=True)[1].str[:-1]
# 替换原数据集中的命中率数据
f_data['FGP'] = round((f_data['FG'] / f_data['FGA']), 3)
f_data['3PP'] = round((f_data['3P'] / f_data['3PA']), 3)
f_data['FTP'] = round((f_data['FT'] / f_data['FTA']), 3)
# 重排数据位置
f_data.insert(2, 'AgeY', f_data.pop('AgeY'))
f_data.insert(6, 'W/L', f_data.pop('W/L'))
f_data.insert(7, 'PTD', f_data.pop('PTD'))
f_data.insert(13, 'FGP', f_data.pop('FGP'))
f_data.insert(17, '3PP', f_data.pop('3PP'))
f_data.insert(21, 'FTP', f_data.pop('FTP'))

f_data['PTD'] = f_data['PTD'].astype(int)
f_data['+/-'] = f_data['+/-'].astype(int)
# 删除冗余数据列
f_data.drop('GR', axis=1, inplace=True)
f_data.drop('FG%', axis=1, inplace=True)
f_data.drop('3P%', axis=1, inplace=True)
f_data.drop('FT%', axis=1, inplace=True)
# 用 0 填充缺失值
f_data.fillna(0, inplace=True)

f_data#.tail(10)

,G,Age,AgeY,Tm,GL,Opp,W/L,PTD,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
Date,,,,,,,,,,,,,,,,,,,,,
1996-11-03,1,18-072,18,LAL,#,MIN,W,6,0,6:22,...,1,1,0,0,1,1,1,0,-1.1,-8
1996-11-05,2,18-074,18,LAL,@,NYK,W,6,0,3:16,...,0,0,0,0,0,1,0,1,-1.1,-8
1996-11-06,3,18-075,18,LAL,@,CHH,L,-10,0,6:49,...,0,0,0,0,0,3,0,5,0.7,0
1996-11-08,4,18-077,18,LAL,@,TOR,L,-1,0,17:15,...,3,3,0,0,1,0,3,10,5.2,2
1996-11-10,5,18-079,18,LAL,#,ATL,W,7,0,8:25,...,2,3,1,0,0,0,0,2,1.9,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-06,62,37-227,37,LAL,#,LAC,L,-10,1,27:46,...,3,3,1,1,0,1,1,17,7.3,-15
2016-04-08,63,37-229,37,LAL,@,NOP,L,-8,1,22:13,...,3,3,4,1,0,2,0,14,7.8,-16
2016-04-10,64,37-231,37,LAL,@,HOU,L,-20,1,27:26,...,2,2,1,2,0,4,1,35,22.5,-10


In [6]:
f_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1346 entries, 1996-11-03 to 2016-04-13
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   G       1346 non-null   object 
 1   Age     1346 non-null   object 
 2   AgeY    1346 non-null   int32  
 3   Tm      1346 non-null   object 
 4   GL      1346 non-null   object 
 5   Opp     1346 non-null   object 
 6   W/L     1346 non-null   object 
 7   PTD     1346 non-null   int32  
 8   GS      1346 non-null   object 
 9   MP      1346 non-null   object 
 10  FG      1346 non-null   int32  
 11  FGA     1346 non-null   int32  
 12  FGP     1346 non-null   float64
 13  3P      1346 non-null   int32  
 14  3PA     1346 non-null   int32  
 15  3PP     1346 non-null   float64
 16  FT      1346 non-null   int32  
 17  FTA     1346 non-null   int32  
 18  FTP     1346 non-null   float64
 19  ORB     1346 non-null   int32  
 20  DRB     1346 non-null   int32  
 21  TRB     1346 non-nu

In [7]:
f_data['MP'].map(lambda x: datetime.strptime(x, "%M:%S"))  # 时间数据类型转换
# f_data

Date
1996-11-03   1900-01-01 00:06:22
1996-11-05   1900-01-01 00:03:16
1996-11-06   1900-01-01 00:06:49
1996-11-08   1900-01-01 00:17:15
1996-11-10   1900-01-01 00:08:25
                     ...        
2016-04-06   1900-01-01 00:27:46
2016-04-08   1900-01-01 00:22:13
2016-04-10   1900-01-01 00:27:26
2016-04-11   1900-01-01 00:18:57
2016-04-13   1900-01-01 00:42:09
Name: MP, Length: 1346, dtype: datetime64[ns]

# Playoffs数据收集

## 网站解析及数据获取

In [8]:
p_list = []
def playoffs():
    url = 'https://www.basketball-reference.com/players/b/bryanko01/gamelog-playoffs/'
    r = requests.get(url, timeout=30)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, "html.parser")
    d_find = soup.find('div', {'id':'div_pgl_basic_playoffs'}).find('table')
    for i in range(1,225):
        hi = []
        tbody = d_find.find('tbody').find('tr', {'id':'pgl_basic_playoffs.'+str(i)})
        if not tbody:
            continue
        for td in tbody.find_all('td', recursive=False):
            ht = td.text
            hi.append(ht)
#             print(ht)
        p_list.append(hi)
    return p_list

playoffs();

In [9]:
j_list = ['G', 'PlayoffsDate', 'Series', 'Tm', 'GL', 'Opp', 'G#', 'GR', 'GS', 'MP', 'FG', 'FGA', 'FG%', 
          '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 
          'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-']


## 数据预处理

In [10]:
# p_list
p_data = pd.DataFrame(p_list, columns=j_list)
p_data['PlayoffsDate'] = p_data['PlayoffsDate'].map(lambda x: datetime.strptime(x, "%Y-%m-%d"))
p_data = p_data.set_index('PlayoffsDate')
p_data[['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']] = p_data[['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].astype(int)

p_data['GL'].replace('', '#', inplace=True)
p_data['W/L'] = p_data['GR'].str.split(' ', expand=True)[0]
p_data['PTD'] = p_data['GR'].str.split('\(', expand=True)[1].str[:-1]
p_data['FGP'] = round((p_data['FG'] / p_data['FGA']), 3)
p_data['3PP'] = round((p_data['3P'] / p_data['3PA']), 3)
p_data['FTP'] = round((p_data['FT'] / p_data['FTA']), 3)

p_data.insert(6, 'W/L', p_data.pop('W/L'))
p_data.insert(7, 'PTD', p_data.pop('PTD'))
p_data.insert(13, 'FGP', p_data.pop('FGP'))
p_data.insert(17, '3PP', p_data.pop('3PP'))
p_data.insert(21, 'FTP', p_data.pop('FTP'))

p_data['PTD'] = p_data['PTD'].astype(int)
p_data['+/-'] = p_data['+/-'].astype(int)

p_data.drop('GR', axis=1, inplace=True)
p_data.drop('FG%', axis=1, inplace=True)
p_data.drop('3P%', axis=1, inplace=True)
p_data.drop('FT%', axis=1, inplace=True)

p_data.fillna(0, inplace=True)

p_data

,G,Series,Tm,GL,Opp,G#,W/L,PTD,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
PlayoffsDate,,,,,,,,,,,,,,,,,,,,,
1997-04-25,1,WC1,LAL,#,POR,1,W,18,0,0:46,...,0,0,0,0,0,0,0,2,1.7,2
1997-04-27,2,WC1,LAL,#,POR,2,W,14,0,4:50,...,0,0,0,0,0,0,1,6,3.9,-4
1997-04-30,3,WC1,LAL,@,POR,3,L,-8,0,26:51,...,4,4,2,1,0,4,5,22,12.5,15
1997-05-02,4,WC1,LAL,@,POR,4,W,4,0,5:34,...,0,0,0,0,0,0,0,0,0.0,-2
1997-05-04,5,WCS,LAL,@,UTA,1,L,-16,0,14:28,...,1,2,3,0,1,0,5,3,-0.1,-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-05-14,8,WCS,LAL,@,OKC,1,L,-29,1,31:55,...,3,3,2,0,0,3,3,20,7.5,-27
2012-05-16,9,WCS,LAL,@,OKC,2,L,-2,1,42:46,...,2,3,4,4,0,2,4,20,10.6,5
2012-05-18,10,WCS,LAL,#,OKC,3,W,3,1,38:58,...,5,7,6,2,0,2,4,36,27.6,11


In [11]:
p_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 220 entries, 1997-04-25 to 2012-05-21
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   G       220 non-null    object 
 1   Series  220 non-null    object 
 2   Tm      220 non-null    object 
 3   GL      220 non-null    object 
 4   Opp     220 non-null    object 
 5   G#      220 non-null    object 
 6   W/L     220 non-null    object 
 7   PTD     220 non-null    int32  
 8   GS      220 non-null    object 
 9   MP      220 non-null    object 
 10  FG      220 non-null    int32  
 11  FGA     220 non-null    int32  
 12  FGP     220 non-null    float64
 13  3P      220 non-null    int32  
 14  3PA     220 non-null    int32  
 15  3PP     220 non-null    float64
 16  FT      220 non-null    int32  
 17  FTA     220 non-null    int32  
 18  FTP     220 non-null    float64
 19  ORB     220 non-null    int32  
 20  DRB     220 non-null    int32  
 21  TRB     220 non-null

# 数据保存

In [12]:
with pd.ExcelWriter(r'kobe.xlsx') as writer:
    f_data.to_excel(writer, sheet_name='Regular')  #保存数据
    p_data.to_excel(writer, sheet_name='Playoffs')

# 数据读取

In [13]:
Regular = pd.read_excel('kobe.xlsx', 'Regular', index_col=0)
Playoffs = pd.read_excel('kobe.xlsx', 'Playoffs', index_col=0)
Regular.head(10)

,G,Age,AgeY,Tm,GL,Opp,W/L,PTD,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
Date,,,,,,,,,,,,,,,,,,,,,
1996-11-03,1,18-072,18,LAL,#,MIN,W,6,0,6:22,...,1,1,0,0,1,1,1,0,-1.1,-8
1996-11-05,2,18-074,18,LAL,@,NYK,W,6,0,3:16,...,0,0,0,0,0,1,0,1,-1.1,-8
1996-11-06,3,18-075,18,LAL,@,CHH,L,-10,0,6:49,...,0,0,0,0,0,3,0,5,0.7,0
1996-11-08,4,18-077,18,LAL,@,TOR,L,-1,0,17:15,...,3,3,0,0,1,0,3,10,5.2,2
1996-11-10,5,18-079,18,LAL,#,ATL,W,7,0,8:25,...,2,3,1,0,0,0,0,2,1.9,-4
1996-11-12,6,18-081,18,LAL,@,HOU,W,11,0,5:03,...,1,1,0,0,0,2,0,2,-0.7,-4
1996-11-13,7,18-082,18,LAL,@,SAS,L,-12,0,6:06,...,0,0,0,1,0,0,2,6,5.6,-4
1996-11-15,8,18-084,18,LAL,#,LAC,W,7,0,7:01,...,0,0,0,0,0,1,4,4,-1.4,1
1996-11-17,9,18-086,18,LAL,@,PHO,W,14,0,14:29,...,0,2,2,0,0,4,2,16,10.4,-3


# Regular数据分析

In [14]:
minutes = Regular['MP'].str.split(':', expand=True)[0].astype(int)
seconds = Regular['MP'].str.split(':', expand=True)[1].astype(int)
total_time = round(minutes + seconds/60, 1)
# round(total_time.sum()/1346, 1)
Regular['MPP'] = total_time

## 各年龄段数据

### 场均贡献

In [15]:
p_age = round(Regular.groupby(['AgeY'])['TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'].mean(), 1)
p_age

,TRB,AST,STL,BLK,TOV,PF,PTS
AgeY,,,,,,,
18,1.9,1.3,0.7,0.3,1.6,1.4,7.6
19,3.1,2.5,0.9,0.5,2.0,2.3,15.4
20,5.3,3.8,1.4,1.0,3.1,3.1,19.9
21,6.3,4.9,1.6,0.9,2.8,3.3,22.5
22,5.9,5.0,1.7,0.6,3.2,3.3,28.5
23,5.5,5.5,1.5,0.4,2.8,2.8,25.2
24,6.9,5.9,2.2,0.8,3.5,2.7,30.0
25,5.5,5.1,1.7,0.4,2.6,2.7,24.0
26,5.9,6.0,1.3,0.8,4.1,2.6,27.6


### 出场数及平均上场时间

In [16]:
g_age = Regular.groupby(['AgeY'])['G'].count()
m_age = round(Regular.groupby(['AgeY'])['MPP'].mean(), 1)
g_age = pd.concat([g_age, m_age], axis=1)
g_age

,G,MPP
AgeY,,
18,71,15.5
19,79,26.1
20,50,37.9
21,66,38.2
22,68,41.0
23,80,38.3
24,82,41.5
25,65,37.6
26,66,40.8


### 首发出场数

In [17]:
gs_age = Regular.groupby(['AgeY'])['GS'].sum()
gs_age

AgeY
18     6
19     1
20    50
21    62
22    68
23    80
24    82
25    64
26    66
27    80
28    77
29    82
30    82
31    73
32    82
33    58
34    78
35     6
36    35
37    66
Name: GS, dtype: int64

### 场均命中率

In [18]:
f_age = round(Regular.groupby(['AgeY'])['FG', 'FGA', '3P', '3PA', 'FT', 'FTA'].mean(), 1)

f_age['FGP'] = round(Regular.groupby(['AgeY'])['FG'].sum()/Regular.groupby(['AgeY'])['FGA'].sum(), 3)
f_age['3PP'] = round(Regular.groupby(['AgeY'])['3P'].sum()/Regular.groupby(['AgeY'])['3PA'].sum(), 3)
f_age['FTP'] = round(Regular.groupby(['AgeY'])['FT'].sum()/Regular.groupby(['AgeY'])['FTA'].sum(), 3)

f_age.insert(2, 'FGP', f_age.pop('FGP'))
f_age.insert(5, '3PP', f_age.pop('3PP'))

f_age

,FG,FGA,FGP,3P,3PA,3PP,FT,FTA,FTP
AgeY,,,,,,,,,
18,2.5,5.9,0.417,0.7,1.9,0.375,1.9,2.3,0.819
19,4.9,11.6,0.428,0.9,2.8,0.341,4.6,5.8,0.794
20,7.2,15.6,0.465,0.5,2.0,0.267,4.9,5.8,0.839
21,8.4,17.9,0.468,0.7,2.2,0.319,5.0,6.1,0.821
22,10.3,22.2,0.464,0.9,2.9,0.305,7.0,8.2,0.853
23,9.4,20.0,0.469,0.4,1.6,0.250,6.1,7.4,0.829
24,10.6,23.5,0.451,1.5,4.0,0.383,7.3,8.7,0.843
25,7.9,18.1,0.438,1.1,3.3,0.327,7.0,8.2,0.852
26,8.7,20.1,0.433,2.0,5.9,0.339,8.2,10.1,0.816


### 合并数据

In [19]:
age_m = pd.concat([g_age, gs_age, f_age, p_age], axis=1)
age_m

,G,MPP,GS,FG,FGA,FGP,3P,3PA,3PP,FT,FTA,FTP,TRB,AST,STL,BLK,TOV,PF,PTS
AgeY,,,,,,,,,,,,,,,,,,,
18,71,15.5,6,2.5,5.9,0.417,0.7,1.9,0.375,1.9,2.3,0.819,1.9,1.3,0.7,0.3,1.6,1.4,7.6
19,79,26.1,1,4.9,11.6,0.428,0.9,2.8,0.341,4.6,5.8,0.794,3.1,2.5,0.9,0.5,2.0,2.3,15.4
20,50,37.9,50,7.2,15.6,0.465,0.5,2.0,0.267,4.9,5.8,0.839,5.3,3.8,1.4,1.0,3.1,3.1,19.9
21,66,38.2,62,8.4,17.9,0.468,0.7,2.2,0.319,5.0,6.1,0.821,6.3,4.9,1.6,0.9,2.8,3.3,22.5
22,68,41.0,68,10.3,22.2,0.464,0.9,2.9,0.305,7.0,8.2,0.853,5.9,5.0,1.7,0.6,3.2,3.3,28.5
23,80,38.3,80,9.4,20.0,0.469,0.4,1.6,0.250,6.1,7.4,0.829,5.5,5.5,1.5,0.4,2.8,2.8,25.2
24,82,41.5,82,10.6,23.5,0.451,1.5,4.0,0.383,7.3,8.7,0.843,6.9,5.9,2.2,0.8,3.5,2.7,30.0
25,65,37.6,64,7.9,18.1,0.438,1.1,3.3,0.327,7.0,8.2,0.852,5.5,5.1,1.7,0.4,2.6,2.7,24.0
26,66,40.8,66,8.7,20.1,0.433,2.0,5.9,0.339,8.2,10.1,0.816,5.9,6.0,1.3,0.8,4.1,2.6,27.6


### 输or赢 & 主or客

#### 各年龄段输赢数据

In [20]:
win_lose_age = Regular.groupby(['AgeY'])['W/L'].value_counts()
win_lose_age

AgeY  W/L
18    W      48
      L      23
19    W      58
      L      21
20    W      31
      L      19
21    W      55
      L      11
22    W      45
      L      23
23    W      56
      L      24
24    W      50
      L      32
25    W      48
      L      17
26    L      38
      W      28
27    W      45
      L      35
28    W      39
      L      38
29    W      57
      L      25
30    W      65
      L      17
31    W      51
      L      22
32    W      57
      L      25
33    W      36
      L      22
34    W      42
      L      36
35    L       4
      W       2
36    L      25
      W      10
37    L      53
      W      13
Name: W/L, dtype: int64

#### 主客场输赢数据

In [21]:
home_away_age = Regular.groupby(['AgeY', 'GL'])['W/L'].value_counts()
home_away_age

AgeY  GL  W/L
18    #   W      24
          L       8
      @   W      24
          L      15
19    #   W      33
                 ..
36    @   W       6
37    #   L      22
          W       8
      @   L      31
          W       5
Name: W/L, Length: 79, dtype: int64

In [22]:
home_away = Regular.groupby(['GL'])['W/L'].value_counts()
home_away

GL  W/L
#   W      481
    L      188
@   W      355
    L      322
Name: W/L, dtype: int64

## 对阵各队数据

### 场均贡献

In [23]:
p_team = round(Regular.groupby(['Opp'])['TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'].mean(), 1)
p_team

,TRB,AST,STL,BLK,TOV,PF,PTS
Opp,,,,,,,
ATL,4.7,4.5,1.6,0.4,2.7,2.2,22.3
BOS,5.1,4.4,1.3,0.5,2.9,3.1,25.1
BRK,5.0,4.0,1.7,0.3,3.7,2.7,21.3
CHA,5.0,4.6,1.0,0.4,3.6,2.9,28.2
CHH,5.9,3.7,1.2,0.8,3.1,2.2,18.3
CHI,5.4,4.5,1.9,0.6,3.5,2.5,23.7
CHO,6.0,3.0,1.0,0.0,2.3,1.3,21.3
CLE,5.6,5.0,1.1,0.5,3.2,2.7,23.3
DAL,6.1,4.5,1.4,0.5,3.3,2.7,24.0


### 出场数及平均上场时间

In [24]:
g_team = Regular.groupby(['Opp'])['G'].count()
m_team = round(Regular.groupby(['Opp'])['MPP'].mean(), 1)
g_team = pd.concat([g_team, m_team], axis=1)
g_team

,G,MPP
Opp,,
ATL,29,34.1
BOS,31,37.0
BRK,3,35.6
CHA,17,39.8
CHH,11,30.4
CHI,31,35.9
CHO,3,29.8
CLE,34,34.3
DAL,60,37.0


### 首发出场数

In [25]:
gs_team = Regular.groupby(['Opp'])['GS'].sum()
gs_team

Opp
ATL    25
BOS    28
BRK     3
CHA    17
CHH     7
CHI    27
CHO     3
CLE    31
DAL    54
DEN    57
DET    29
GSW    58
HOU    56
IND    31
LAC    61
MEM    49
MIA    29
MIL    29
MIN    56
NJN    20
NOH    28
NOK     7
NOP     6
NYK    30
OKC    22
ORL    25
PHI    28
PHO    58
POR    56
SAC    56
SAS    56
SEA    36
TOR    29
UTA    52
VAN    11
WAS    28
WSB     0
Name: GS, dtype: int64

### 场均命中率

In [26]:
f_team = round(Regular.groupby(['Opp'])['FG', 'FGA', '3P', '3PA', 'FT', 'FTA'].mean(), 1)

f_team['FGP'] = round(Regular.groupby(['Opp'])['FG'].sum()/Regular.groupby(['Opp'])['FGA'].sum(), 3)
f_team['3PP'] = round(Regular.groupby(['Opp'])['3P'].sum()/Regular.groupby(['Opp'])['3PA'].sum(), 3)
f_team['FTP'] = round(Regular.groupby(['Opp'])['FT'].sum()/Regular.groupby(['Opp'])['FTA'].sum(), 3)

f_team.insert(2, 'FGP', f_team.pop('FGP'))
f_team.insert(5, '3PP', f_team.pop('3PP'))

f_team

,FG,FGA,FGP,3P,3PA,3PP,FT,FTA,FTP
Opp,,,,,,,,,
ATL,8.1,17.9,0.453,1.2,3.8,0.324,4.8,6.2,0.782
BOS,8.8,21.0,0.419,1.4,4.3,0.318,6.1,7.6,0.804
BRK,7.3,18.3,0.400,0.7,3.3,0.200,6.0,7.0,0.857
CHA,10.1,22.9,0.440,1.8,5.6,0.326,6.3,7.4,0.849
CHH,6.8,15.7,0.434,0.3,1.9,0.143,4.4,4.9,0.889
CHI,8.6,19.7,0.439,1.0,3.7,0.261,5.5,7.6,0.723
CHO,6.7,19.3,0.345,2.7,8.3,0.320,5.3,6.0,0.889
CLE,8.1,18.3,0.441,1.2,3.5,0.342,6.0,7.2,0.836
DAL,8.3,18.6,0.447,1.1,3.5,0.308,6.3,7.6,0.834


### 合并数据

In [27]:
team_m = pd.concat([g_team, gs_team, f_team, p_team], axis=1)
team_m

,G,MPP,GS,FG,FGA,FGP,3P,3PA,3PP,FT,FTA,FTP,TRB,AST,STL,BLK,TOV,PF,PTS
Opp,,,,,,,,,,,,,,,,,,,
ATL,29,34.1,25,8.1,17.9,0.453,1.2,3.8,0.324,4.8,6.2,0.782,4.7,4.5,1.6,0.4,2.7,2.2,22.3
BOS,31,37.0,28,8.8,21.0,0.419,1.4,4.3,0.318,6.1,7.6,0.804,5.1,4.4,1.3,0.5,2.9,3.1,25.1
BRK,3,35.6,3,7.3,18.3,0.400,0.7,3.3,0.200,6.0,7.0,0.857,5.0,4.0,1.7,0.3,3.7,2.7,21.3
CHA,17,39.8,17,10.1,22.9,0.440,1.8,5.6,0.326,6.3,7.4,0.849,5.0,4.6,1.0,0.4,3.6,2.9,28.2
CHH,11,30.4,7,6.8,15.7,0.434,0.3,1.9,0.143,4.4,4.9,0.889,5.9,3.7,1.2,0.8,3.1,2.2,18.3
CHI,31,35.9,27,8.6,19.7,0.439,1.0,3.7,0.261,5.5,7.6,0.723,5.4,4.5,1.9,0.6,3.5,2.5,23.7
CHO,3,29.8,3,6.7,19.3,0.345,2.7,8.3,0.320,5.3,6.0,0.889,6.0,3.0,1.0,0.0,2.3,1.3,21.3
CLE,34,34.3,31,8.1,18.3,0.441,1.2,3.5,0.342,6.0,7.2,0.836,5.6,5.0,1.1,0.5,3.2,2.7,23.3
DAL,60,37.0,54,8.3,18.6,0.447,1.1,3.5,0.308,6.3,7.6,0.834,6.1,4.5,1.4,0.5,3.3,2.7,24.0


### 输or赢 & 主or客

#### 对阵各队输赢数据

In [28]:
win_lose_team = Regular.groupby(['Opp'])['W/L'].value_counts()
win_lose_team

Opp  W/L
ATL  W      18
     L      11
BOS  W      18
     L      13
BRK  W       3
            ..
VAN  W      17
     L       1
WAS  W      20
     L      11
WSB  W       1
Name: W/L, Length: 72, dtype: int64

#### 主客场输赢数据

In [29]:
home_away_team = Regular.groupby(['Opp', 'GL'])['W/L'].value_counts()
home_away_team

Opp  GL  W/L
ATL  #   W      10
         L       3
     @   L       8
         W       8
BOS  #   W       8
                ..
WAS  #   W      10
         L       4
     @   W      10
         L       7
WSB  #   W       1
Name: W/L, Length: 141, dtype: int64

## 各年龄段数据总和

In [30]:
s_age = Regular.groupby(['AgeY'])['TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'].sum()
s_age = pd.concat([Regular.groupby(['AgeY'])['G'].count(), s_age], axis=1)
s_age

,G,TRB,AST,STL,BLK,TOV,PF,PTS
AgeY,,,,,,,,
18,71,132,91,49,23,112,102,539
19,79,242,199,74,40,157,180,1220
20,50,264,190,72,50,157,153,996
21,66,416,323,106,62,182,220,1485
22,68,399,338,114,43,220,222,1938
23,80,441,438,118,35,223,228,2019
24,82,564,481,181,67,288,218,2461
25,65,359,330,112,28,171,176,1557
26,66,392,398,86,53,270,174,1819


## 对阵各队数据总和

In [31]:
s_team = Regular.groupby(['Opp'])['TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'].sum()
s_team = pd.concat([Regular.groupby(['Opp'])['G'].count(), s_team], axis=1)
s_team

,G,TRB,AST,STL,BLK,TOV,PF,PTS
Opp,,,,,,,,
ATL,29,136,130,47,12,78,64,646
BOS,31,158,135,41,15,91,96,777
BRK,3,15,12,5,1,11,8,64
CHA,17,85,78,17,6,61,50,480
CHH,11,65,41,13,9,34,24,201
CHI,31,167,141,58,20,109,78,736
CHO,3,18,9,3,0,7,4,64
CLE,34,192,169,39,16,108,92,793
DAL,60,367,269,84,28,196,163,1441


## 最大输赢分差

In [32]:
Regular['PTD'] = Regular['PTD'].astype(int)
Regular['+/-'] = Regular['+/-'].astype(int)

### 年龄

In [33]:
sp_age = pd.DataFrame(Regular.groupby(['AgeY'])['PTD'].max()).rename(columns={'PTD':'MAX'})
pn_age = pd.DataFrame(Regular.groupby(['AgeY'])['+/-'].max()).rename(columns={'+/-':'P_MAX'})

sp_age = pd.concat([Regular.groupby(['AgeY'])['PTD'].min(), sp_age, Regular.groupby(['AgeY'])['+/-'].min(), pn_age], axis=1)
sp_age = sp_age.rename(columns={'PTD':'MIN', '+/-':'N_MIN'})
sp_age

,MIN,MAX,N_MIN,P_MAX
AgeY,,,,
18,-26,30,-18,23
19,-21,39,-16,43
20,-27,28,-24,27
21,-24,46,-20,40
22,-33,37,-25,34
23,-18,41,-19,40
24,-27,28,-22,37
25,-22,46,-26,33
26,-27,23,-30,26


In [34]:
with pd.ExcelWriter(r'Regular_Age.xlsx') as writer1:
    age_m.to_excel(writer1, sheet_name='data_age')
    win_lose_age.to_excel(writer1, sheet_name='win_lose_age')
    home_away_age.to_excel(writer1, sheet_name='home_away_age')
    home_away.to_excel(writer1, sheet_name='home_away')
    sp_age.to_excel(writer1, sheet_name='dff_point')
    s_age.to_excel(writer1, sheet_name='age_sum')

### 球队

In [35]:
sp_team = pd.DataFrame(Regular.groupby(['Opp'])['PTD'].max()).rename(columns={'PTD':'MAX'})
pn_team = pd.DataFrame(Regular.groupby(['Opp'])['+/-'].max()).rename(columns={'+/-':'P_MAX'})

sp_team = pd.concat([Regular.groupby(['Opp'])['PTD'].min(), sp_team, Regular.groupby(['Opp'])['+/-'].min(), pn_team], axis=1)
sp_team = sp_team.rename(columns={'PTD':'MIN', '+/-':'N_MIN'})
sp_team

,MIN,MAX,N_MIN,P_MAX
Opp,,,,
ATL,-17,46,-15,33
BOS,-21,26,-29,20
BRK,5,9,-6,20
CHA,-20,33,-18,14
CHH,-10,41,-15,27
CHI,-23,28,-22,22
CHO,-19,15,-13,7
CLE,-19,55,-19,46
DAL,-36,35,-29,39


In [36]:
with pd.ExcelWriter(r'Regular_Team.xlsx') as writer2:
    team_m.to_excel(writer2, sheet_name='data_team')
    win_lose_team.to_excel(writer2, sheet_name='win_lose_team')
    home_away_team.to_excel(writer2, sheet_name='home_away_team')
    home_away.to_excel(writer2, sheet_name='home_away')
    sp_team.to_excel(writer2, sheet_name='dff_point')
    s_team.to_excel(writer2, sheet_name='team_sum')

## 数据总和

In [37]:
Regular.head()

,G,Age,AgeY,Tm,GL,Opp,W/L,PTD,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,MPP
Date,,,,,,,,,,,,,,,,,,,,,
1996-11-03,1,18-072,18,LAL,#,MIN,W,6,0,6:22,...,1,0,0,1,1,1,0,-1.1,-8,6.4
1996-11-05,2,18-074,18,LAL,@,NYK,W,6,0,3:16,...,0,0,0,0,1,0,1,-1.1,-8,3.3
1996-11-06,3,18-075,18,LAL,@,CHH,L,-10,0,6:49,...,0,0,0,0,3,0,5,0.7,0,6.8
1996-11-08,4,18-077,18,LAL,@,TOR,L,-1,0,17:15,...,3,0,0,1,0,3,10,5.2,2,17.2
1996-11-10,5,18-079,18,LAL,#,ATL,W,7,0,8:25,...,3,1,0,0,0,0,2,1.9,-4,8.4


In [38]:
regular_sum = Regular[['TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].sum()
regular_sum

TRB     7047
AST     6306
STL     1944
BLK      640
TOV     4010
PF      3353
PTS    33643
dtype: int64

# Playoffs数据分析

In [39]:
Playoffs.head(5)

,G,Series,Tm,GL,Opp,G#,W/L,PTD,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
PlayoffsDate,,,,,,,,,,,,,,,,,,,,,
1997-04-25,1,WC1,LAL,#,POR,1,W,18,0,0:46,...,0,0,0,0,0,0,0,2,1.7,2
1997-04-27,2,WC1,LAL,#,POR,2,W,14,0,4:50,...,0,0,0,0,0,0,1,6,3.9,-4
1997-04-30,3,WC1,LAL,@,POR,3,L,-8,0,26:51,...,4,4,2,1,0,4,5,22,12.5,15
1997-05-02,4,WC1,LAL,@,POR,4,W,4,0,5:34,...,0,0,0,0,0,0,0,0,0.0,-2
1997-05-04,5,WCS,LAL,@,UTA,1,L,-16,0,14:28,...,1,2,3,0,1,0,5,3,-0.1,-10


In [40]:
p_minutes = Playoffs['MP'].str.split(':', expand=True)[0].astype(int)
p_seconds = Playoffs['MP'].str.split(':', expand=True)[1].astype(int)
p_total_time = round(p_minutes + p_seconds/60, 1)
# round(p_total_time.sum()/1346, 1)
Playoffs['MPP'] = p_total_time

## 各回合数据

### 场均贡献

In [41]:
p_rd = round(Playoffs.groupby(['Series'])['TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'].mean(), 1)
p_rd

,TRB,AST,STL,BLK,TOV,PF,PTS
Series,,,,,,,
FIN,5.7,5.1,1.8,0.9,3.3,3.2,25.3
WC1,4.7,4.9,1.4,0.5,2.9,3.0,25.2
WCF,5.4,5.3,1.2,0.9,2.6,3.1,26.8
WCS,4.9,4.0,1.3,0.5,3.0,2.9,25.6


### 出场数及平均上场时间

In [42]:
g_rd = Playoffs.groupby(['Series'])['G'].count()
m_rd = round(Playoffs.groupby(['Series'])['MPP'].mean(), 1)
g_rd = pd.concat([g_rd, m_rd], axis=1)
g_rd

,G,MPP
Series,,
FIN,37,42.7
WC1,74,38.2
WCF,45,40.5
WCS,64,37.7


### 首发出场数

In [43]:
gs_rd = Playoffs.groupby(['Series'])['GS'].sum()
gs_rd

Series
FIN    37
WC1    66
WCF    41
WCS    56
Name: GS, dtype: int64

### 场均命中率

In [44]:
f_rd = round(Playoffs.groupby(['Series'])['FG', 'FGA', '3P', '3PA', 'FT', 'FTA'].mean(), 1)

f_rd['FGP'] = round(Playoffs.groupby(['Series'])['FG'].sum()/Playoffs.groupby(['Series'])['FGA'].sum(), 3)
f_rd['3PP'] = round(Playoffs.groupby(['Series'])['3P'].sum()/Playoffs.groupby(['Series'])['3PA'].sum(), 3)
f_rd['FTP'] = round(Playoffs.groupby(['Series'])['FT'].sum()/Playoffs.groupby(['Series'])['FTA'].sum(), 3)

f_rd.insert(2, 'FGP', f_rd.pop('FGP'))
f_rd.insert(5, '3PP', f_rd.pop('3PP'))

f_rd

,FG,FGA,FGP,3P,3PA,3PP,FT,FTA,FTP
Series,,,,,,,,,
FIN,9.0,21.8,0.412,1.3,4.1,0.314,6.0,7.1,0.848
WC1,9.0,20.1,0.447,1.5,4.1,0.362,5.7,7.0,0.820
WCF,9.6,20.5,0.469,1.6,4.3,0.363,6.0,7.4,0.811
WCS,9.1,20.1,0.455,1.0,3.6,0.276,6.3,7.9,0.799


### 合并数据

In [45]:
rd_m = pd.concat([g_rd, gs_rd, f_rd, p_rd], axis=1)
rd_m

,G,MPP,GS,FG,FGA,FGP,3P,3PA,3PP,FT,FTA,FTP,TRB,AST,STL,BLK,TOV,PF,PTS
Series,,,,,,,,,,,,,,,,,,,
FIN,37,42.7,37,9.0,21.8,0.412,1.3,4.1,0.314,6.0,7.1,0.848,5.7,5.1,1.8,0.9,3.3,3.2,25.3
WC1,74,38.2,66,9.0,20.1,0.447,1.5,4.1,0.362,5.7,7.0,0.820,4.7,4.9,1.4,0.5,2.9,3.0,25.2
WCF,45,40.5,41,9.6,20.5,0.469,1.6,4.3,0.363,6.0,7.4,0.811,5.4,5.3,1.2,0.9,2.6,3.1,26.8
WCS,64,37.7,56,9.1,20.1,0.455,1.0,3.6,0.276,6.3,7.9,0.799,4.9,4.0,1.3,0.5,3.0,2.9,25.6


### 输or赢 & 主or客

#### 各回合输赢数据

In [46]:
win_lose_rd = Playoffs.groupby(['Series'])['W/L'].value_counts()
win_lose_rd

Series  W/L
FIN     W      23
        L      14
WC1     W      50
        L      24
WCF     W      28
        L      17
WCS     W      34
        L      30
Name: W/L, dtype: int64

#### 主客场输赢数据

In [47]:
home_away_rd = Playoffs.groupby(['Series', 'GL'])['W/L'].value_counts()
home_away_rd

Series  GL  W/L
FIN     #   W      14
            L       4
        @   L      10
            W       9
WC1     #   W      34
            L       5
        @   L      19
            W      16
WCF     #   W      17
            L       6
        @   L      11
            W      11
WCS     #   W      23
            L       9
        @   L      21
            W      11
Name: W/L, dtype: int64

In [48]:
rd_home_away = Playoffs.groupby(['GL'])['W/L'].value_counts()
rd_home_away

GL  W/L
#   W      88
    L      24
@   L      61
    W      47
Name: W/L, dtype: int64

## 对阵各队数据

### 场均贡献

In [49]:
pl_team = round(Playoffs.groupby(['Opp'])['TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'].mean(), 1)
pl_team

,TRB,AST,STL,BLK,TOV,PF,PTS
Opp,,,,,,,
BOS,6.5,4.4,2.4,0.5,3.8,3.5,27.2
DAL,3.0,2.5,1.8,0.2,2.8,1.8,23.2
DEN,5.2,5.6,1.1,0.6,2.8,2.9,31.9
DET,2.8,4.4,1.8,0.6,3.6,3.0,22.6
HOU,5.8,5.0,2.2,1.1,1.8,2.6,24.2
IND,4.6,4.2,1.0,1.4,1.2,3.6,15.6
MIN,4.6,6.1,1.4,0.2,2.7,2.4,28.1
NJN,5.8,5.2,1.5,0.8,3.8,2.8,26.8
NOH,3.7,3.8,1.5,0.3,3.3,2.7,22.5


### 出场数及平均上场时间

In [50]:
gl_team = Playoffs.groupby(['Opp'])['G'].count()
ml_team = round(Playoffs.groupby(['Opp'])['MPP'].mean(), 1)
gl_team = pd.concat([gl_team, ml_team], axis=1)
gl_team

,G,MPP
Opp,,
BOS,13,42.0
DAL,4,37.1
DEN,17,40.6
DET,5,46.2
HOU,16,39.8
IND,5,35.1
MIN,12,43.7
NJN,4,43.5
NOH,6,34.4


### 首发出场数

In [51]:
gsl_team = Playoffs.groupby(['Opp'])['GS'].sum()
gsl_team

Opp
BOS    13
DAL     4
DEN    17
DET     5
HOU    16
IND     5
MIN    12
NJN     4
NOH     6
OKC    11
ORL     5
PHI     5
PHO    23
POR    13
SAC    16
SAS    30
SEA     0
UTA    15
Name: GS, dtype: int64

### 场均命中率

In [52]:
fl_team = round(Playoffs.groupby(['Opp'])['FG', 'FGA', '3P', '3PA', 'FT', 'FTA'].mean(), 1)

fl_team['FGP'] = round(Playoffs.groupby(['Opp'])['FG'].sum()/Playoffs.groupby(['Opp'])['FGA'].sum(), 3)
fl_team['3PP'] = round(Playoffs.groupby(['Opp'])['3P'].sum()/Playoffs.groupby(['Opp'])['3PA'].sum(), 3)
fl_team['FTP'] = round(Playoffs.groupby(['Opp'])['FT'].sum()/Playoffs.groupby(['Opp'])['FTA'].sum(), 3)

fl_team.insert(2, 'FGP', fl_team.pop('FGP'))
fl_team.insert(5, '3PP', fl_team.pop('3PP'))

fl_team

,FG,FGA,FGP,3P,3PA,3PP,FT,FTA,FTP
Opp,,,,,,,,,
BOS,9.2,22.6,0.405,1.8,5.8,0.320,7.1,8.4,0.844
DAL,9.5,20.8,0.458,1.2,5.5,0.227,3.0,3.8,0.800
DEN,11.1,23.5,0.471,2.1,6.1,0.346,7.6,9.1,0.844
DET,8.6,22.6,0.381,0.8,4.6,0.174,4.6,5.0,0.920
HOU,8.7,20.5,0.424,1.2,3.9,0.306,5.6,6.4,0.882
IND,6.6,18.0,0.367,0.4,2.0,0.200,2.0,2.2,0.909
MIN,9.7,22.8,0.423,1.5,4.8,0.310,7.2,9.0,0.806
NJN,9.0,17.5,0.514,1.5,2.8,0.545,7.2,9.0,0.806
NOH,7.5,17.2,0.437,1.2,3.2,0.368,6.3,7.7,0.826


### 合并数据

In [53]:
team_ml = pd.concat([gl_team, gsl_team, fl_team, pl_team], axis=1)
team_ml

,G,MPP,GS,FG,FGA,FGP,3P,3PA,3PP,FT,FTA,FTP,TRB,AST,STL,BLK,TOV,PF,PTS
Opp,,,,,,,,,,,,,,,,,,,
BOS,13,42.0,13,9.2,22.6,0.405,1.8,5.8,0.320,7.1,8.4,0.844,6.5,4.4,2.4,0.5,3.8,3.5,27.2
DAL,4,37.1,4,9.5,20.8,0.458,1.2,5.5,0.227,3.0,3.8,0.800,3.0,2.5,1.8,0.2,2.8,1.8,23.2
DEN,17,40.6,17,11.1,23.5,0.471,2.1,6.1,0.346,7.6,9.1,0.844,5.2,5.6,1.1,0.6,2.8,2.9,31.9
DET,5,46.2,5,8.6,22.6,0.381,0.8,4.6,0.174,4.6,5.0,0.920,2.8,4.4,1.8,0.6,3.6,3.0,22.6
HOU,16,39.8,16,8.7,20.5,0.424,1.2,3.9,0.306,5.6,6.4,0.882,5.8,5.0,2.2,1.1,1.8,2.6,24.2
IND,5,35.1,5,6.6,18.0,0.367,0.4,2.0,0.200,2.0,2.2,0.909,4.6,4.2,1.0,1.4,1.2,3.6,15.6
MIN,12,43.7,12,9.7,22.8,0.423,1.5,4.8,0.310,7.2,9.0,0.806,4.6,6.1,1.4,0.2,2.7,2.4,28.1
NJN,4,43.5,4,9.0,17.5,0.514,1.5,2.8,0.545,7.2,9.0,0.806,5.8,5.2,1.5,0.8,3.8,2.8,26.8
NOH,6,34.4,6,7.5,17.2,0.437,1.2,3.2,0.368,6.3,7.7,0.826,3.7,3.8,1.5,0.3,3.3,2.7,22.5


### 输or赢 & 主or客

#### 对阵各队输赢数据

In [54]:
lose_win_team = Playoffs.groupby(['Opp'])['W/L'].value_counts()
lose_win_team

Opp  W/L
BOS  L       7
     W       6
DAL  L       4
DEN  W      12
     L       5
DET  L       4
     W       1
HOU  W      11
     L       5
IND  W       4
     L       1
MIN  W       8
     L       4
NJN  W       4
NOH  W       4
     L       2
OKC  L       6
     W       5
ORL  W       4
     L       1
PHI  W       4
     L       1
PHO  W      12
     L      11
POR  W      16
     L       5
SAC  W      11
     L       5
SAS  W      18
     L      12
SEA  W       2
     L       1
UTA  W      13
     L      11
Name: W/L, dtype: int64

#### 主客场输赢数据

In [55]:
away_home_team = Playoffs.groupby(['Opp', 'GL'])['W/L'].value_counts()
away_home_team

Opp  GL  W/L
BOS  #   W      5
         L      2
     @   L      5
         W      1
DAL  #   L      2
               ..
SEA  @   W      1
UTA  #   W      9
         L      3
     @   L      8
         W      4
Name: W/L, Length: 63, dtype: int64

## 各回合数据总和

In [56]:
s_rd = Playoffs.groupby(['Series'])['TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'].sum()
s_rd = pd.concat([Playoffs.groupby(['Series'])['G'].count(), s_rd], axis=1)
s_rd

,G,TRB,AST,STL,BLK,TOV,PF,PTS
Series,,,,,,,,
FIN,37,211,187,65,33,123,118,937
WC1,74,350,359,107,39,217,219,1863
WCF,45,243,237,55,40,116,138,1204
WCS,64,315,257,83,32,191,185,1636


## 对阵各队数据总和

In [57]:
sl_team = Playoffs.groupby(['Opp'])['TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'].sum()
sl_team = pd.concat([Playoffs.groupby(['Opp'])['G'].count(), sl_team], axis=1)
sl_team

,G,TRB,AST,STL,BLK,TOV,PF,PTS
Opp,,,,,,,,
BOS,13,84,57,31,6,50,46,354
DAL,4,12,10,7,1,11,7,93
DEN,17,88,95,19,11,47,50,542
DET,5,14,22,9,3,18,15,113
HOU,16,92,80,36,18,29,41,387
IND,5,23,21,5,7,6,18,78
MIN,12,55,73,17,2,32,29,337
NJN,4,23,21,6,3,15,11,107
NOH,6,22,23,9,2,20,16,135


## 最大输赢分差

### 回合

In [58]:
sp_rd = pd.DataFrame(Playoffs.groupby(['Series'])['PTD'].max()).rename(columns={'PTD':'MAX'})
pn_rd = pd.DataFrame(Playoffs.groupby(['Series'])['+/-'].max()).rename(columns={'+/-':'P_MAX'})

sp_rd = pd.concat([Playoffs.groupby(['Series'])['PTD'].min(), sp_rd, Playoffs.groupby(['Series'])['+/-'].min(), pn_rd], axis=1)
sp_rd = sp_rd.rename(columns={'PTD':'MIN', '+/-':'N_MIN'})
sp_rd

,MIN,MAX,N_MIN,P_MAX
Series,,,,
FIN,-39,25,-35,25
WC1,-31,30,-28,29
WCF,-35,39,-28,31
WCS,-36,40,-29,31


In [59]:
with pd.ExcelWriter(r'Playoffs_Round.xlsx') as writer3:
    rd_m.to_excel(writer3, sheet_name='data_round')
    win_lose_rd.to_excel(writer3, sheet_name='win_lose_round')
    home_away_rd.to_excel(writer3, sheet_name='home_away_round')
    rd_home_away.to_excel(writer3, sheet_name='round_home_away')
    sp_rd.to_excel(writer3, sheet_name='dff_point')
    s_rd.to_excel(writer3, sheet_name='round_sum')

### 球队

In [60]:
spl_team = pd.DataFrame(Playoffs.groupby(['Opp'])['PTD'].max()).rename(columns={'PTD':'MAX'})
pnl_team = pd.DataFrame(Playoffs.groupby(['Opp'])['+/-'].max()).rename(columns={'+/-':'P_MAX'})

spl_team = pd.concat([Playoffs.groupby(['Opp'])['PTD'].min(), spl_team, Playoffs.groupby(['Opp'])['+/-'].min(), pnl_team], axis=1)
spl_team = spl_team.rename(columns={'PTD':'MIN', '+/-':'N_MIN'})
spl_team

,MIN,MAX,N_MIN,P_MAX
Opp,,,,
BOS,-39,22,-35,15
DAL,-36,-2,-29,6
DEN,-19,27,-28,31
DET,-20,8,-20,7
HOU,-15,40,-12,30
IND,-33,17,-22,15
MIN,-28,30,-26,29
NJN,3,23,1,20
NOH,-9,18,-1,15


In [61]:
with pd.ExcelWriter(r'Playoffs_Team.xlsx') as writer4:
    team_ml.to_excel(writer4, sheet_name='data_team')
    lose_win_team.to_excel(writer4, sheet_name='win_lose_team')
    away_home_team.to_excel(writer4, sheet_name='home_away_team')
    rd_home_away.to_excel(writer4, sheet_name='home_away')
    spl_team.to_excel(writer4, sheet_name='dff_point')
    sl_team.to_excel(writer4, sheet_name='team_sum')